<a href="https://colab.research.google.com/github/AbhinandanRoy7/AbhinandanRoy7/blob/main/Speech_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!pip install yt-dlp
!pip install librosa soundfile
!pip install transformers torch
!pip install librosa soundfile transformers torch noisereduce
!pip install pyctcdecode
!pip install transformers

In [29]:
!yt-dlp -x --audio-format wav https://www.youtube.com/shorts/oyPn7DnRVqY -o "audio.%(ext)s"

[youtube] Extracting URL: https://www.youtube.com/shorts/oyPn7DnRVqY
[youtube] oyPn7DnRVqY: Downloading webpage
[youtube] oyPn7DnRVqY: Downloading ios player API JSON
[youtube] oyPn7DnRVqY: Downloading web creator player API JSON
[youtube] oyPn7DnRVqY: Downloading m3u8 information
[info] oyPn7DnRVqY: Downloading 1 format(s): 251
[download] Destination: audio.webm
[download] 100% of  285.39KiB in 00:00:00 at 973.70KiB/s
[ExtractAudio] Destination: audio.wav
Deleting original file audio.webm (pass -k to keep)


In [30]:
import librosa
import soundfile as sf
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch
import librosa
import numpy as np
import noisereduce as nr

In [31]:
audio_file = "audio.wav"
speech, sr = librosa.load(audio_file, sr=16000)  # Resample to 16kHz

# If the audio is stereo, convert it to mono
if speech.ndim > 1:
    speech = librosa.to_mono(speech)


# Load audio
audio, sr = librosa.load('audio.wav', sr=16000)

# Reduce noise
reduced_noise_audio = nr.reduce_noise(y=audio, sr=sr)

# Save the processed audio
sf.write('processed_audio.wav', speech, sr)


# Save the preprocessed audio
sf.write("processed_audio.wav", speech, 16000)

In [32]:
!pip install transformers torch librosa
import transformers
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch
import librosa

print("Transformers version:", transformers.__version__)
print("Torch version:", torch.__version__)
print("Librosa version:", librosa.__version__)


Transformers version: 4.44.2
Torch version: 2.4.1+cu121
Librosa version: 0.10.2.post1


In [33]:
kenlm_model_path="path/to/language_model.binary"

In [34]:
# Install required packages
!pip install transformers torch librosa

from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch
import librosa


# Load pre-trained model and processor from Hugging Face
model_name = "facebook/wav2vec2-base-960h"
try:
    processor = Wav2Vec2Processor.from_pretrained(model_name)
    model = Wav2Vec2ForCTC.from_pretrained(model_name)
except Exception as e:
    print("Error loading model or processor:", e)

# Load the preprocessed audio file
speech, sr = librosa.load("processed_audio.wav", sr=16000)

# Trim the audio to the first 10 seconds (or any desired length)
max_length = 10
max_samples = max_length * sr
speech = speech[:max_samples]  # Trimming the audio

# Process the audio input for the model
input_values = processor(speech, return_tensors="pt", sampling_rate=16000).input_values

# Check if input values are not too large
print(f"Input values shape: {input_values.shape}")

# Perform inference (forward pass) with error handling
try:
    with torch.no_grad():  # Avoid storing gradients
        logits = model(input_values).logits

    # Get the predicted tokens
    predicted_ids = torch.argmax(logits, dim=-1)

    # Decode the predicted tokens into text (transcript)
    transcription = processor.decode(predicted_ids[0])

    # Print the transcript
    print("Transcript: ", transcription)

except Exception as e:
    print("Error during inference:", e)


Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

Input values shape: torch.Size([1, 160000])
Transcript:  THIS IS DASH AND HE'S AN ARCHER TRAINING HIS PULL BACK STRENGTH WITH HIS BOW THE AMOUNT OF FORCE NEEDED TO PULL BACK BOWS LIKE THIS ARE A LOT SO TRAINING IS NECESSARY AND HERE DASH IS SHOWING HOW MUCH POUNDS OF FORCE THIS BOW


In [35]:
import numpy as np

# Function to split audio into chunks (e.g., 10 seconds)
def split_audio(audio, sr, chunk_length=10):
    chunk_size = chunk_length * sr
    return [audio[i:i + chunk_size] for i in range(0, len(audio), chunk_size)]

# Split audio into 10-second chunks
chunks = split_audio(speech, sr, chunk_length=10)

# Transcribe each chunk
transcriptions = []
for chunk in chunks:
    input_values = processor(chunk, return_tensors="pt", sampling_rate=16000).input_values
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0])
    transcriptions.append(transcription)

# Combine the transcriptions
full_transcript = " ".join(transcriptions)


In [36]:
import numpy as np
import torch
import librosa
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

# Load pre-trained model and processor from Hugging Face
model_name = "facebook/wav2vec2-base-960h"  # Use a small model for efficiency
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)

# Load the preprocessed audio file
speech, sr = librosa.load("processed_audio.wav", sr=16000)

# Function to split audio into chunks with overlap
def split_audio(audio, sr, chunk_length=10, overlap=2):
    chunk_size = chunk_length * sr
    overlap_size = overlap * sr
    chunks = []
    for i in range(0, len(audio), chunk_size - overlap_size):
        chunks.append(audio[i:i + chunk_size])
    return chunks

# Split audio into 10-second chunks with 2 seconds of overlap
chunks = split_audio(speech, sr, chunk_length=10, overlap=2)

# Transcribe each chunk
transcriptions = []
for i, chunk in enumerate(chunks):
    # Process the audio input for the model
    input_values = processor(chunk, return_tensors="pt", sampling_rate=16000).input_values

    # Perform inference (forward pass)
    with torch.no_grad():  # Avoid storing gradients
        logits = model(input_values).logits

    # Get the predicted tokens
    predicted_ids = torch.argmax(logits, dim=-1)

    # Decode the predicted tokens into text (transcript)
    transcription = processor.decode(predicted_ids[0])

    # Print progress
    print(f"Processed chunk {i + 1}/{len(chunks)}")

    transcriptions.append(transcription)

# Combine the transcriptions into a single full transcript
full_transcript = " ".join(transcriptions)
print("Full Transcript: ", full_transcript)



Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

Processed chunk 1/3
Processed chunk 2/3
Processed chunk 3/3
Full Transcript:  THIS IS DASH AND HE'S AN ARCHER TRAINING HIS PULL BACK STRENGTH WITH HIS BOW THE AMOUNT OF FORCE NEEDED TO PULL BACK BOWS LIKE THIS ARE A LOT SO TRAINING IS NECESSARY AND HERE DASH IS SHOWING HOW MUCH POUNDS OF FORCE THIS BOW SHES SHOWING HOW MUCH POUNDS OF FORCE THIS BEAW REALLY IS COMING IN AT ONE HUNDRED FIFTY EIGHT POUNDS AND HEAR'S WHAT HE LOOKS LIKE PULLING BACK THAT SAME EXACT BOW MAYBE BEING AN ARCHER IS THE BEST WAY TO GET A PHYSIQUE LIKE THAT THE BEST WAY TO GET A PHYSIQUE LIKE THAT
